<a href="https://colab.research.google.com/github/ritaja2003/Predictive-Analysis/blob/main/Predictive_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-learn pandas matplotlib seaborn tensorflow keras


In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ritajachatterjee2003","key":"b725e3b5c916cf6c751628dbedac07f4"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d shivamb/machine-predictive-maintenance-classification
!unzip machine-predictive-maintenance-classification.zip


Dataset URL: https://www.kaggle.com/datasets/shivamb/machine-predictive-maintenance-classification
License(s): CC0-1.0
  0% 0.00/137k [00:00<?, ?B/s]
100% 137k/137k [00:00<00:00, 131MB/s]
Archive:  machine-predictive-maintenance-classification.zip
  inflating: predictive_maintenance.csv  


In [ ]:
# Load & Explore the Data

import pandas as pd

# Load the dataset
df = pd.read_csv("predictive_maintenance.csv")

# Display the first few rows
df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


In [ ]:
import os
print(os.getcwd())


/content


In [ ]:
"""Dataset Overview
The dataset contains:

Type: Type of equipment
Air temperature, Process temperature: Sensor data
Rotational speed, Torque, Tool wear: Operational conditions
Failure Type: Label (whether failure occurred & its type)"""

print(df.isnull().sum())

df.fillna(df.select_dtypes(include=['number']).mean(), inplace=True)



UDI                        0
Product ID                 0
Type                       0
Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Tool wear [min]            0
Target                     0
Failure Type               0
dtype: int64


In [ ]:
if 'Type' in df.columns:
    df = pd.get_dummies(df, columns=['Type'], drop_first=True)
else:
    print("Column 'Type' not found in dataset.")




In [ ]:
if 'Failure Type' in df.columns:
    df['Failure'] = df['Failure Type'].apply(lambda x: 1 if x != 'No Failure' else 0)
    df.drop(columns=['Failure Type'], inplace=True)
else:
    print("Column 'Failure Type' not found in dataset.")



In [ ]:
print(df.dtypes)
df = pd.get_dummies(df, drop_first=True)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# Select only numeric columns for scaling
numeric_columns = df.select_dtypes(include=['number']).columns
df_scaled = pd.DataFrame(scaler.fit_transform(df[numeric_columns]), columns=numeric_columns)

# If you need to keep non-numeric columns, merge them back after scaling
df_scaled['Failure'] = df['Failure']





UDI                          int64
Product ID                  object
Air temperature [K]        float64
Process temperature [K]    float64
Rotational speed [rpm]       int64
Torque [Nm]                float64
Tool wear [min]              int64
Target                       int64
Type_L                        bool
Type_M                        bool
Failure                      int64
dtype: object


In [ ]:
from sklearn.model_selection import train_test_split

X = df_scaled.drop(columns=['Failure'])
y = df_scaled['Failure']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
#Train a random forest classifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Train Random Forest model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predictions
y_pred = clf.predict(X_test)

# Evaluate Model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.996
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1935
           1       0.97      0.91      0.94        65

    accuracy                           1.00      2000
   macro avg       0.98      0.95      0.97      2000
weighted avg       1.00      1.00      1.00      2000



In [ ]:
#Train a Deep Learning Model (LSTM)

import numpy as np

# Reshape data to fit LSTM input (samples, time steps, features)
X_train_reshaped = np.reshape(X_train.values, (X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = np.reshape(X_test.values, (X_test.shape[0], X_test.shape[1], 1))


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Define LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    LSTM(50),
    Dense(1, activation='sigmoid')  # Binary Classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, validation_data=(X_test_reshaped, y_test))


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9655 - loss: 0.2755 - val_accuracy: 0.9675 - val_loss: 0.1427
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9636 - loss: 0.1363 - val_accuracy: 0.9675 - val_loss: 0.0699
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9776 - loss: 0.0536 - val_accuracy: 0.9960 - val_loss: 0.0299
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9980 - loss: 0.0198 - val_accuracy: 0.9960 - val_loss: 0.0276
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9975 - loss: 0.0202 - val_accuracy: 0.9960 - val_loss: 0.0261
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9977 - loss: 0.0153 - val_accuracy: 0.9960 - val_loss: 0.0256
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9970 - loss: 0.0194 - val_accuracy: 0.9960 - val_loss: 0.0275
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9975 - loss: 0.0154 - val_accuracy: 0.9960 - val_

In [ ]:
model.save("predictive_maintenance_model.h5")
from google.colab import files
files.download("predictive_maintenance_model.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install flask flask-cors joblib pandas numpy scikit-learn


In [ ]:
import joblib

# Assuming your trained model is in a variable called 'model'
joblib.dump(model, "ml_model.pkl")
print("Model saved!")


Model saved!


In [ ]:
!pip install streamlit



In [ ]:
# Create the Streamlit app file
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import joblib
import numpy as np

# Load the trained model
model = joblib.load("ml_model.pkl")  # Update with your model path

# Streamlit app
st.title("Machine Learning Model Prediction")
st.write("Enter the input features for prediction:")

# Define the number of features your model expects
num_features = 5  # Change this to the actual number of features your model requires

# Input fields for features
features = []
for i in range(num_features):
    feature = st.number_input(f"Feature {i + 1}")
    features.append(feature)

# Button to make prediction
if st.button("Predict"):
    input_data = np.array(features).reshape(1, -1)
    prediction = model.predict(input_data)
    st.write(f"Prediction: {int(prediction[0])}")
    """)


In [ ]:
import streamlit as st
import joblib
import numpy as np

# Load the trained model
model = joblib.load("ml_model.pkl")  # Update with your model path

# Streamlit app
st.title("Machine Learning Model Prediction")
st.write("Enter the input features for prediction:")

# Define the number of features your model expects
num_features = 5  # Change this to the actual number of features your model requires

# Input fields for features
features = []
for i in range(num_features):
    feature = st.number_input(f"Feature {i + 1}")
    features.append(feature)

# Button to make prediction
if st.button("Predict"):
    input_data = np.array(features).reshape(1, -1)
    prediction = model.predict(input_data)
    st.write(f"Prediction: {int(prediction[0])}")




/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
2025-02-18 16:19:32.350 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-18 16:19:32.889 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-02-18 16:19:32.900 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-18 16:19:32.904 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-18 16:19:32.908 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-18 16:19:32.911

In [ ]:
!pip install streamlit pyngrok joblib numpy pandas scikit-learn


In [ ]:
%%writefile app.py
import streamlit as st
import joblib
import numpy as np

# Load the trained model
model = joblib.load("ml_model.pkl")  # Make sure this file is uploaded in Colab

# Streamlit app
st.title("Machine Learning Model Prediction")
st.write("Enter the input features for prediction:")

# Define the number of features your model expects
num_features = 5  # Change this to match your model

# Input fields for features
features = []
for i in range(num_features):
    feature = st.number_input(f"Feature {i + 1}")
    features.append(feature)

# Button to make prediction
if st.button("Predict"):
    input_data = np.array(features).reshape(1, -1)
    prediction = model.predict(input_data)
    st.write(f"Prediction: {int(prediction[0])}")


Overwriting app.py


In [ ]:
!ls


app.py						   predictive_maintenance.csv
machine-predictive-maintenance-classification.zip  predictive_maintenance_model.h5
ml_model.pkl					   sample_data


In [ ]:
!streamlit run app.py --server.port 8501 &

from pyngrok import ngrok
public_url = ngrok.connect(port='8501')
print(f"Streamlit App URL: {public_url}")





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.139.123.203:8501



In [ ]:
!pip install pyngrok
!ngrok authtoken 2tDljjaBlsR0W8iNpX1qVCdPkCk_64MWdgDnAjrZHViXPMGCJ  # Replace YOUR_NGROK_AUTHTOKEN with your actual token


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:


!pip install pyngrok
from pyngrok import ngrok
import os

# Start Streamlit in the background
os.system("streamlit run app.py &")

# Start ngrok with proper configuration
public_url = ngrok.connect(8501, "http")
print(f"Streamlit App URL: {public_url}")


Streamlit App URL: NgrokTunnel: "https://fa30-34-139-123-203.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!pip install streamlit pyngrok


In [ ]:
%%writefile app.py
import streamlit as st
import joblib
import numpy as np

# Load the trained model
model = joblib.load("ml_model.pkl")  # Ensure this file exists

# Streamlit app
st.title("Machine Learning Model Prediction")
st.write("Enter the input features for prediction:")

# Define the number of features your model expects
num_features = 5  # Change this based on your model

# Input fields for features
features = []
for i in range(num_features):
    feature = st.number_input(f"Feature {i + 1}")
    features.append(feature)

# Button to make prediction
if st.button("Predict"):
    input_data = np.array(features).reshape(1, -1)
    prediction = model.predict(input_data)
    st.write(f"Prediction: {int(prediction[0])}")


Overwriting app.py


In [ ]:
import os
from pyngrok import ngrok

# Kill any previous Streamlit instances (just in case)
os.system("kill -9 $(lsof -t -i:8501)")

# Run Streamlit in the background
os.system("streamlit run app.py &")

# Start ngrok tunnel
public_url = ngrok.connect(8501, "http")
print(f"Streamlit App is Live at: {public_url}")


Streamlit App is Live at: NgrokTunnel: "https://01a5-34-139-123-203.ngrok-free.app" -> "http://localhost:8501"
